In [ ]:
from os.path import join

from hydromt_wflow import WflowModel
from hydromt_sfincs import SfincsModel
from hydromt.config import configread
from hydromt.log import setuplog
import yaml

# output dir and area of interest
bbox = [34.4, -20.05, 34.93, -19.45]

## Introduction

In this notebook we build a coupled Wflow - Sfincs model in two steps.

1. We setup the SFINCS model and automatically determine the locations where rivers flow into the model domain.  
2. We setup a Wflow model containing the catchments of the rivers flowing into the SFINCS model and add model output locations matching the SFINCS discharge boundary locations.

The models are build from global data sets which are shown below for both models.

## 1. Build SFINCS model

In [ ]:
# initialize SfincsModel class 
sf = SfincsModel(
    root=r'../models/sfincs', # output dir
    mode='w+', # open in (over)write mode
    data_libs=[r'../data/sfincs_data/data_catalog.yml'],  # link to data catalog
    logger=setuplog('sfincs'), 
)

In [ ]:
# overview of data used for setting up the static SFINCS layers
sf_data = ['fabdem', 'merit_hydro', 'vito']
data_cols = ['category', 'paper_doi', 'paper_ref', 'source_url', 'source_version','source_license','notes']
sf.data_catalog.to_dataframe().loc[sf_data, data_cols]

In [ ]:
# inspect model building process
sf_config = configread('sfincs_build.yml')
print(sf_config.keys())

In [ ]:
sf.build(opt=sf_config,region={'bbox': bbox})

NOTE that the steps above can also be done from the command line interface with

```
hydromt build sfincs ../models/sfincs -i sfincs_build.yml -r "{'bbox': [34.4,-20.05,34.93,-19.45]}" -d ../data/sfincs_data/data_catalog.yml -v --fo
```

In [ ]:
# plot the SFINCS dep map and boundaries 
sf.grid['dep'].attrs.update(long_name="elevation", units="m")
fig, ax = sf.plot_basemap(figsize=(8,6))
fig.savefig(join(sf.root, 'sfincs_basemap.png'), dpi=450)

## 2. Build Wflow model

In [ ]:
# initialize WflowModel class model
wf = WflowModel(
    root = r'../models/wflow', # output dir
    mode='w+', # open in (over)write mode
    data_libs=[r'../data/wflow_data/data_catalog.yml'], # link to data catalog
    logger=setuplog('wflow'), 
)

In [ ]:
# overview of data used for setting up the static SFINCS layers
wf_data = ['merit_hydro_ihu', 'merit_hydro_ihu_index', 'rivers_lin2019_v1', 'hydro_reservoirs', 'hydro_lakes', 'soilgrids', 'modis_lai', 'globcover', 'vito']
wf.data_catalog.to_dataframe().loc[wf_data, data_cols]

### 2A Build base model

In [ ]:
# read config file with step how to build the model
wf_config = configread('wflow_build.yml')
print(wf_config.keys())

In [ ]:
# we add a setup gauges step to get output at the Sfincs model inflow locations
wf_config['setup_gauges'] = {
    'gauges_fn': join(sf.root, 'gis', 'src.geojson'),
    'snap_uparea': True, 
    'index_col': 'index',
    'basename': 'sfincs',
    'gauge_toml_header': ['Q'],  # save discharge only on these locations
    'gauge_toml_param': ['lateral.river.q_av'],
    'derive_subcatch': True,
}

In [ ]:
# outlets: True to get rivers that have an oulet whithin the bbox (only for coastal regions)
wf.build(opt=wf_config,region={'basin': bbox, 'outlets': True})

# alternatively, you can provide a GeoDataFrame (or vector file) with the outlet locations to derive subbasins
# the uparea argument is added to snap the outlet locations to nearest downstream river
# gdf_src = sf.forcing['dis'].vector.to_gdf()
# region={'subbasin': gdf_src, 'uparea': gdf_src['uparea'].values.min()}

### 2B update static model layers based on different landuse

In [ ]:
# update model with different landuse for a sensitivity analysis8

wf.setup_lulcmaps(
    lulc_fn='vito'
)
wf.setup_config(**{'input.path_static': 'staticmaps_vito.nc'})
wf.write_staticmaps()

### 2C plot model base maps

In [ ]:
from wflow_utils import plot_basemap
# plot the wflow model
fig, ax = plot_basemap(wf, figsize=(8,6))
sf.grid.raster.box.to_crs(wf.crs).boundary.plot(ax=ax, color='r', lw=1, ls='--', label='SFINCS domain')
fig.savefig(join(wf.root, 'wflow_basemap.png'), dpi=450)